# import package

In [1]:
%matplotlib inline

import numpy as np
from numpy import nan
import pandas as pd
from pandas import (DataFrame, date_range)
import matplotlib.pyplot as plt
import alphalens
import datetime
from datetime import date
import copy

from alphalens.tears import (create_returns_tear_sheet,
                      create_information_tear_sheet,
                      create_turnover_tear_sheet,
                      create_summary_tear_sheet,
                      create_full_tear_sheet,
                      create_event_returns_tear_sheet,
                      create_event_study_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

# The main function of Alphalens is to surface the most relevant statistics and plots about an alpha factor, including:

# Returns Analysis
# Information Coefficient Analysis
# Turnover Analysis
# Grouped Analysis

# Read Stock Data

In [2]:
infile = open('/Users/arthur/Desktop/factor_investment/2005.csv', 'r')
df = pd.read_csv(infile, low_memory=False)

In [3]:
col_name = ['coid','stockName','date','open_adj','high_adj','low_adj','close_adj','volume_d','volume_m','return','overturn','SharesOutstanding','marketValue','最後顯示買價','最後顯示賣價','log_return','marketValueRatio','成交值比重','成交筆數','本益比-TSE','本益比-TEJ','股價淨值比-TSE','股價淨值比-TEJ','漲跌停','股價營收比-TEJ','股利殖利率-TSE','dividendRate','股價漲跌','高低價差比','次日開盤參考價','次日漲停價','次日跌停價','注意股票','處置股票','全額交割','市場別']
df.columns = col_name

# Overview & Observe Stock Data

In [4]:
df.shape

(6938880, 36)

In [5]:
df.head(10)

,coid,stockName,date,open_adj,high_adj,low_adj,close_adj,volume_d,volume_m,return,...,dividendRate,股價漲跌,高低價差比,次日開盤參考價,次日漲停價,次日跌停價,注意股票,處置股票,全額交割,市場別
0,0050,���j�x�W50,2005/01/03,26.7005,26.7667,26.5352,26.6124,5175,250369,-0.4124,...,NaN,-0.20,0.8660,48.30,51.65,44.92,NaN,NaN,NaN,TSE
1,1101,�x�d,2005/01/03,6.3797,6.6239,6.3797,6.5323,36056,769907,2.3923,...,1.57,0.50,3.8278,21.40,22.80,20.00,NaN,NaN,NaN,TSE
2,1102,�Ȫd,2005/01/03,6.0344,6.1422,6.0075,6.1152,13519,304980,1.3393,...,2.69,0.30,2.2321,22.70,24.20,21.20,NaN,NaN,NaN,TSE
3,1103,�Ūd,2005/01/03,9.4391,9.6827,9.3782,9.5609,7326,114613,1.2903,...,0.00,0.20,3.2258,15.70,16.70,14.65,NaN,NaN,NaN,TSE
4,1104,���d,2005/01/03,4.6624,4.7341,4.6265,4.7341,4696,61621,1.9305,...,0.00,0.25,2.3166,13.20,14.10,12.30,NaN,NaN,NaN,TSE
5,1107,�إx,2005/01/03,1.1600,1.1600,1.1600,1.1600,0,0,0.0000,...,0.00,0.00,0.0000,0.70,NaN,NaN,NaN,NaN,NaN,TSE
6,1108,����,2005/01/03,5.6335,5.8114,5.6039,5.7818,4454,43265,2.6316,...,1.54,0.25,3.6842,9.75,10.40,9.10,NaN,NaN,NaN,TSE
7,1109,�H�j,2005/01/03,5.8792,5.9605,5.8521,5.9063,755,8241,1.8692,...,2.67,0.20,1.8692,10.90,11.65,10.15,NaN,NaN,NaN,TSE
8,1110,�F�d,2005/01/03,6.9114,7.1013,6.7975,6.9874,7582,69487,2.2222,...,0.00,0.20,4.4444,9.20,9.80,8.60,NaN,NaN,NaN,TSE
9,1201,����,2005/01/03,10.4745,11.0258,10.4321,10.8138,7531,95711,3.6585,...,0.00,0.45,5.6911,12.75,13.60,11.90,NaN,NaN,NaN,TSE


In [6]:
# Column處置股票的值

df.iloc[list(df.iloc[:, 33].isna() == False), 33]

425        D
3169       D
4507       D
5844       D
7180       D
          ..
6936292    D
6937672    D
6937952    D
6938350    D
6938546    D
Name: 處置股票, Length: 29517, dtype: object

In [7]:
# 各column缺值有多少個

# df.isna().sum()

# Narrow Down to Certain Stocks

In [8]:
# header $ index_col

df_valid = pd.read_csv('/Users/arthur/Desktop/factor_investment/StockValidation.csv', index_col=0)
# df_valid.iloc[0, 0] = '0050'
# df_valid.iloc[1, 0] = '0056'
df_valid = df_valid.drop([0, 1])
print(df_valid.head(5))

# turn type(coid) from int to str
df_valid.iloc[:, 0] = [str(x) for x in df_valid.iloc[:, 0]]

print('1101' in np.array(df_valid.iloc[:, 0]))

   stock_id  date_start    date_end
2      1101  2005-01-03  2021-08-26
3      1102  2005-01-03  2021-08-26
4      1103  2005-01-03  2021-08-26
5      1104  2005-01-03  2021-08-26
6      1108  2005-01-03  2021-08-26
True


In [9]:
#  df.iloc[:, 0] == '1101' |  df.iloc[:, 0] == '1102'
# select_index = df.iloc[:, 0] == '1101'

In [10]:
# for i in df_valid.iloc[:, 0]:
#     print(i)
#     if i == '1201':
#         break
#     select

In [12]:
[i for i, x in enumerate(df.iloc[:, 0] == '1101') if x][0:10]

[1, 1343, 2684, 4023, 5361, 6696, 8031, 9367, 10704, 12039]

In [95]:
df_stock = pd.DataFrame(columns = df.columns)

for i in df_valid.iloc[:, 0]:
    print(i)

    if i == '1201':
        break
    
    for j in [k for k, m in enumerate(df.iloc[:, 0] == i) if m]:
        df_stock.loc[df_stock.shape[0]] = df.loc[j]

1101
1102
1103
1104
1108
1109
1110
1201


In [96]:
# df.iloc[:, 0] in df_valid.iloc[:, 0]
print(type(df.iloc[:, 0]))
print(type(df_valid.iloc[:, 0]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [103]:
df_stock

,coid,stockName,date,open_adj,high_adj,low_adj,close_adj,volume_d,volume_m,return,...,dividendRate,股價漲跌,高低價差比,次日開盤參考價,次日漲停價,次日跌停價,注意股票,處置股票,全額交割,市場別
0,1101,�x�d,2005/01/03,6.3797,6.6239,6.3797,6.5323,36056,769907,2.3923,...,1.57,0.5,3.8278,21.4,22.8,20.0,NaN,NaN,NaN,TSE
1,1101,�x�d,2005/01/04,6.5323,6.5323,6.4407,6.4407,10024,212677,-1.4019,...,1.6,-0.3,1.4019,21.1,22.5,19.7,NaN,NaN,NaN,TSE
2,1101,�x�d,2005/01/05,6.3797,6.4102,6.2881,6.4102,9348,194943,-0.4739,...,1.6,-0.1,1.8957,21.0,22.4,19.6,NaN,NaN,NaN,TSE
3,1101,�x�d,2005/01/06,6.3492,6.4102,6.3186,6.3492,2815,58707,-0.9524,...,1.62,-0.2,1.4286,20.8,22.2,19.4,NaN,NaN,NaN,TSE
4,1101,�x�d,2005/01/07,6.3492,6.3797,6.105,6.105,15895,323818,-3.8462,...,1.68,-0.8,4.3269,20.0,21.4,18.6,NaN,NaN,NaN,TSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28758,1110,�F�d,2021/08/20,16.8,17.0,16.6,16.6,296,4993,-0.8955,...,0.6024,-0.15,2.3881,16.6,18.25,14.95,NaN,NaN,NaN,TSE
28759,1110,�F�d,2021/08/23,16.6,17.0,16.6,17.0,209,3513,2.4096,...,0.5882,0.4,2.4096,17.0,18.7,15.3,NaN,NaN,NaN,TSE
28760,1110,�F�d,2021/08/24,16.95,17.55,16.95,17.55,237,4090,3.2353,...,0.5698,0.55,3.5294,17.55,19.3,15.8,NaN,NaN,NaN,TSE
28761,1110,�F�d,2021/08/25,17.4,17.85,17.4,17.85,124,2185,1.7094,...,0.5602,0.3,2.5641,17.85,19.6,16.1,NaN,NaN,NaN,TSE


# Read Financial Report Data

In [13]:
# infile = open('/Users/arthur/Desktop/factor_investment/2022.csv', 'r', encoding='BIG5-HKSCS')
# df = pd.read_csv(infile, sep=',', encoding_errors='ignore')
# col_name = ['coid','stockName','date','open_adj','high_adj','low_adj','close_adj','volume_d','volume_m','return','overturn','SharesOutstanding','marketValue','最後顯示買價','最後顯示賣價','log_return','marketValueRatio','成交值比重','成交筆數','本益比-TSE','本益比-TEJ','股價淨值比-TSE','股價淨值比-TEJ','漲跌停','股價營收比-TEJ','股利殖利率-TSE','dividendRate','股價漲跌','高低價差比','次日開盤參考價','次日漲停價','次日跌停價','注意股票','處置股票','全額交割','市場別']
infile = open('/Users/arthur/Desktop/factor_investment/2022_utf8.csv', 'r')
df_2 = pd.read_csv(infile, low_memory=False)

# Overview & Observe Financial Report Data

In [14]:
df_2.head(5)

,證券代碼,年月,季別,合併(Y/N),月份,幣別,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,...,財報發布日,財報類別（1個別2個體3合併）,財報年月起日,財報年月迄日,市場別,交易所主產業代碼,交易所子產業代碼,TEJ主產業代碼,TEJ子產業代碼,財報附註TEJ是否完成Y/N
0,1101 台泥,2021/12,4,Y,12,NTD,91121942.0,363813.0,6387543.0,15508688.0,...,2022/02/25,3.0,2021/01,2021/12,TSE,M1100,M1100,11A,11A,Y
1,1110 東泥,2021/12,4,Y,12,NTD,419504.0,248685.0,0.0,0.0,...,2022/03/18,3.0,2021/01,2021/12,TSE,M1100,M1100,11A,11A,N
2,1216 統一,2021/12,4,Y,12,NTD,80837880.0,10394193.0,0.0,15300236.0,...,2022/03/09,3.0,2021/01,2021/12,TSE,M1200,M1200,12C,12C,Y
3,1229 聯華,2021/12,4,Y,12,NTD,2785515.0,103474.0,666216.0,0.0,...,2022/03/15,3.0,2021/01,2021/12,TSE,M1200,M1200,12B,12B,Y
4,1231 聯華食,2021/12,4,Y,12,NTD,327180.0,297223.0,0.0,0.0,...,2022/03/10,3.0,2021/01,2021/12,TSE,M1200,M1200,12Z,12Z,Y


In [15]:
df_2.tail(10).loc[:, ["證券代碼", "年月", "每股現金流量"]]

,證券代碼,年月,每股現金流量
120963,9978 長榮空廚,2005/06,1.08
120964,9980 東森媒體科技,2005/06,0.04
120965,9982 三冠王,2005/06,8.64
120966,9983 永佳樂,2005/06,0.22
120967,9984 全聯有線,2005/06,0.67
120968,9985 觀天下,2005/06,0.67
120969,9986 南天有線,2005/06,1.96
120970,9989 天外天興業,2005/06,2.21
120971,A00010 富邦投信,2005/06,2.34
120972,A00038 富達投信,2005/06,-0.35


In [16]:
# for i in df_2.columns.values:
#     print(i)

# Narrow Down to Certain Stocks

In [17]:
# pip install regex
import re

In [18]:
# select_index_1 = np.array([False]*df_2.shape[0])
# select_index_2 = np.array([])

# for i in df_valid.iloc[:, 0]:
#     select_index_2 = np.array([])
#     print(i)
    
#     if i == '1201':
#         break
    
#     for j in df_2.iloc[:, 0]:
#         select_index_2 = np.append(select_index_2, i in j)
    
#     select_index_1 = np.logical_or(select_index_1, select_index_2)

In [19]:
# 較差的演算法，速度較慢

# df_report = pd.DataFrame(columns = df_2.columns)

# for i in [j for j, k in enumerate(select_index_1) if k]:
#     df_report.loc[df_report.shape[0]] = df_2.loc[i]

In [ ]:
df_report = pd.DataFrame(columns = df_2.columns)

for i in df_valid.iloc[:, 0]:
    print(i)

#     if i == '1201':
#         break
    
    for j in [k for k, m in enumerate([ i in l for l in df_2.iloc[:, 0] ]) if m]:
        df_report.loc[df_report.shape[0]] = df_2.loc[j]

1101
1102
1103
1104
1108
1109
1110
1201
1203
1210
1215
1216
1217
1218
1219
1225
1227
1229
1231
1232
1233
1234
1256
1259
1262
1264
1301
1303
1304
1305
1307
1308
1309
1310
1311
1312
1313
1314
1315
1319
1321
1325
1326
1333
1337
1338
1339
1340
1402
1409
1416
1419
1434
1436
1437
1440
1442
1444
1447
1451
1455
1459
1460
1464
1471
1473
1476
1477
1503
1504
1507
1513
1515
1519
1520
1521
1522
1523
1525
1527
1530
1531
1532
1535
1536
1537
1538
1558
1560
1563
1565
1569
1580
1582
1583
1589
1590
1597
1598
1604
1605
1606
1608
1609
1611
1612
1614
1618
1626
1701
1702
1704
1707
1708
1709
1710
1711
1712
1714
1715
1716
1717
1718
1720
1722
1723
1725
1726
1729
1733
1734
1736
1737
1760
1762
1773
1785
1789
1795
1802
1806
1808
1810
1815
1902
1903
1904
1905
1907
1909
2002
2006
2007
2008
2009
2010
2012
2013
2014
2015
2017
2020
2022
2023
2025
2027
2028
2029
2030
2031
2033
2034
2035
2038
2049
2059
2062
2072
2101
2102
2103
2104
2105
2106
2107
2108
2109
2114
2201
2204
2206
2207
2208
2211
2227
2228
2231
2239
2247
2301


In [ ]:
df_report

# 在df增加年月

In [20]:
for i in range(df.shape[0]):
    df.loc[i, "年月"] = df.loc[i, "date"][0:7]

In [21]:
# df_stock.groupby(["年月", 'coid']).mean()

# 算各股票變異數

In [59]:
pd.DataFrame(df.groupby('coid').var()).iloc[:, 3]

coid
0050     632.227648
0051      64.170770
0052     594.898450
0053     151.618653
0054      22.170584
           ...     
9958    1274.200754
9960      80.482030
9961      12.726700
9962       8.326749
9965       1.900864
Name: close_adj, Length: 2935, dtype: float64

In [145]:
# df_report.groupby(['年月', '證券代碼']).mean()

In [70]:
print(df_2.iloc[:, 6], '\n\n\n')
print(df_2.loc[:, "  現金及約當現金"], '\n\n\n')
print(df_2[["  現金及約當現金"]], '\n\n\n')
print(df_2.loc[:, "每股現金流量"], '\n\n\n')

0         91121942.0
1           419504.0
2         80837880.0
3          2785515.0
4           327180.0
             ...    
120968       91212.0
120969       49042.0
120970      111207.0
120971     1050144.0
120972      459519.0
Name:   現金及約當現金, Length: 120973, dtype: float64 



0         91121942.0
1           419504.0
2         80837880.0
3          2785515.0
4           327180.0
             ...    
120968       91212.0
120969       49042.0
120970      111207.0
120971     1050144.0
120972      459519.0
Name:   現金及約當現金, Length: 120973, dtype: float64 



           現金及約當現金
0       91121942.0
1         419504.0
2       80837880.0
3        2785515.0
4         327180.0
...            ...
120968     91212.0
120969     49042.0
120970    111207.0
120971   1050144.0
120972    459519.0

[120973 rows x 1 columns] 



0         3.09
1         0.47
2         9.92
3         1.37
4         5.11
          ... 
120968    0.67
120969    1.96
120970    2.21
120971    2.34
120972   -0.35
Name: 每股現金

# 計算IR, IC

In [22]:
price_index = pd.Index(date_range(start='2005-6', end='2021-8'))
tickers = np.unique(np.array(df_stock.iloc[:, 0]))
prices = pd.DataFrame(index=price_index, columns=tickers)

NameError: name 'df_stock' is not defined

In [ ]:
for i in price_index:
    for j in tickers:
        prices.loc[i, j] = mean(df.iloc[df.loc[:, 'coid']==j & df.loc[:, '年月'], ])

In [146]:
price = pd.DataFrame(index=)

for i in range(df_report.shape[0]):
    df_report.loc[i, "price"]

,證券代碼,年月,季別,合併(Y/N),月份,幣別,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,...,財報發布日,財報類別（1個別2個體3合併）,財報年月起日,財報年月迄日,市場別,交易所主產業代碼,交易所子產業代碼,TEJ主產業代碼,TEJ子產業代碼,財報附註TEJ是否完成Y/N
0,1101 台泥,2021/12,4,Y,12,NTD,91121942.0,363813.0,6387543.0,15508688.0,...,2022/02/25,3.0,2021/01,2021/12,TSE,M1100,M1100,11A,11A,Y
1,1101 台泥,2021/09,3,Y,9,NTD,59958809.0,335314.0,6180994.0,15157115.0,...,2021/11/11,3.0,2021/01,2021/09,TSE,M1100,M1100,11A,11A,Y
2,1101 台泥,2021/06,2,Y,6,NTD,63835177.0,362554.0,6097715.0,7593330.0,...,2021/08/12,3.0,2021/01,2021/06,TSE,M1100,M1100,11A,11A,Y
3,1101 台泥,2021/03,1,Y,3,NTD,58958539.0,344694.0,5721861.0,9459089.0,...,2021/05/12,3.0,2021/01,2021/03,TSE,M1100,M1100,11A,11A,Y
4,1101 台泥,2020/12,4,Y,12,NTD,51433522.0,353986.0,5596471.0,15769277.0,...,2021/03/19,3.0,2020/01,2020/12,TSE,M1100,M1100,11A,11A,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,1110 東泥,2007/06,2,Y,6,NTD,391180.0,128268.0,0.0,0.0,...,2007/08/27,3.0,2007/01,2007/06,TSE,M1100,M1100,11A,11A,Y
425,1110 東泥,2006/12,4,Y,12,NTD,340303.0,136550.0,0.0,0.0,...,2007/04/25,3.0,2006/01,2006/12,TSE,M1100,M1100,11A,11A,Y
426,1110 東泥,2006/06,2,Y,6,NTD,292032.0,141751.0,0.0,0.0,...,2006/08/28,3.0,2006/01,2006/06,TSE,M1100,M1100,11A,11A,Y
427,1110 東泥,2005/12,4,Y,12,NTD,324397.0,173263.0,0.0,0.0,...,2007/04/25,3.0,2005/01,2005/12,TSE,M1100,M1100,11A,11A,Y
